In [124]:

import pandas as pd
import numpy as np
import os
import sklearn
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.metrics import r2_score, f1_score
# --------cross-validation
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
# -------- classification
import sklearn
from sklearn import neighbors, naive_bayes, svm
# *** SVM classifier
from sklearn.svm import SVC
# --------  metrics:
from sklearn import metrics
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.metrics import make_scorer
from sklearn.ensemble import RandomForestClassifier, VotingClassifier

In [126]:
#split the data to train and test
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2)

In [125]:

def load_dataset(file_name, target_column):
    df=pd.read_csv(file_name)
    TRAINING_FEATURES = df.columns[(df.columns != target_column) & (df.columns!='SKU')& (df.columns!='Item Name')]

    X = df[TRAINING_FEATURES]
    y = df[target_column]
    return X, y

file_name='../EDA/SheinData.csv'
X,y=load_dataset(file_name, 'Classification')

In [127]:
#random forest model
def find_best_random_forest_num_estimators(X_train, y_train):
    parameters = {'n_estimators':[15,10] }
    rf = RandomForestClassifier()
    clf = GridSearchCV(rf, parameters,scoring=make_scorer(metrics.f1_score, greater_is_better=True))
    #fit
    clf.fit(X_train, y_train)
    return clf

#predict
Random_Forest=find_best_random_forest_num_estimators(X_train, y_train)

In [128]:
print(f1_score(y_test,Random_Forest.predict(X_test)))

0.8951612903225806


In [129]:
#svm model
clf=svm.SVC(probability=True)
#fit(train)
clf.fit(X_train, y_train)

#predict
predict=clf.predict(X_test)

#evaluate
print(f1_score(y_test,predict))


0.8070562293274531


In [130]:
#logistic regration model

#fit(train)
def train_by_logistic(X_train, y_train):
    trained_LogisticRegression=LogisticRegression(max_iter=10000,solver="lbfgs")
    trained_LogisticRegression.fit(X_train, y_train)
    return trained_LogisticRegression

logistic_model=train_by_logistic(X_train, y_train)

#predict
def predict_by_logistic(trained_2nd_model, X_test):
    predicted_vals=trained_2nd_model.predict(X_test)
    return predicted_vals

predict_model=predict_by_logistic(logistic_model, X_test)

#evaluate
def evaluate_performance_by_logistic(y_test,y_predicted):
    evaluate_value=f1_score(y_test, y_predicted)
    return evaluate_value

print(evaluate_performance_by_logistic(y_test,predict_model))

0.788627935723115


In [131]:
#ensembel
trained_LogisticRegression=LogisticRegression(max_iter=10000,solver="lbfgs")
rf = RandomForestClassifier(n_estimators=50, random_state=1)
clf=svm.SVC(probability=True)    
est_Ensemble = VotingClassifier(estimators=[('SVM', clf),
                                            ('Logistic Regression', trained_LogisticRegression),
                                            ('Random Forest', rf)],
                                voting='soft',
                                weights=[1,1,1])
est_Ensemble .fit(X_train, y_train)
y_predicted=est_Ensemble.predict(X_test)
print(f1_score(y_test, y_predicted))

0.8847583643122676


In [148]:
df=pd.read_csv('../Wow effect/Shein_Wow_Data.csv')
df2=df.copy()


In [149]:
missing_cols = set( X_train.columns ) - set(df.columns )
for c in missing_cols:
    df[c] = 0
df = df[X_train.columns]
TRAINING_FEATURES = df.columns[(df.columns!='SKU')& (df.columns!='Item Name')]

y_predicted=est_Ensemble.predict(df[TRAINING_FEATURES])

In [150]:
df2['Model Prediction']=y_predicted

In [152]:
df2.to_csv("Wow Prediction.csv" ,index=False, encoding='utf-8-sig')